# Abstract

Implicitly defined, continuous, differentiable signal representations parameterized by NN -> powerful, offering many possible benefits over conventional representations.
**However**, current architectures -> fail to represent a signal's spatial and temporal derivatives
- propose to leverage periodic activation functions for INRs
- dubbed sinusoidal representation networks (**SIRENs**), represent well on complex natural signals and their derivatives.

leveraged to Boundary value problems, such as particular Eikonal equations, Poisson equation, Helmholtz and wave equations.

# Introduction
$$F(x,\phi, \nabla_x\phi,\nabla_x^2\phi, ...) = 0, ~~ \phi : x \mapsto \phi(x)$$

$x \in \R^m$  x: 시-공간 좌표, $\phi$ 는 x를 $\phi(x)$ 로 매핑하는 임의의 함수라고 할때, 우리는 **$\phi$ 의 x에 대한 미분, 혹은 x와 $\phi$를 입력으로 받아 0을 반환하는 함수 F.**

따라서, $\phi$ 를 매개변수화 하는 신경망을 학습하는 것이 목표.